ESM1v (MAC)
- python3.11 -m venv esm1v
- activate it 
- python -m pip install --upgrade pip setuptools wheel
- python -m pip install torch fair-esm ipykernel

ESM1v (PC/CUDA)
winget install Python.Python.3.11
python -m pip install --upgrade pip setuptools wheel
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip install fair-esm ipykernel numpy psutil
python -m ipykernel install --user --name esm1v_cuda --display-name "esm1v_cuda (RTX3060)" 
#made the venv visible to notebooks, VS Code can now attach to it. before that step  no Jupyter kernel entry point. 

NOTES:
* on multiple proces, cannot load ESM-1v once and “share it” across workers, CUDA contexts are per process, each OS process has its own CUDA context. With spawn (required for CUDA on Windows):workers do not inherit Python objects. On Linux you could theoretically:	fork()inherit memory pagesshare CPU tensors (copy-on-write). But CUDA + fork = undefined behavior. 

* track gpu usage while esm1v script running: nvidia-smi dmon -s pucvmt

* GPU 2.50/2.58 GB | CPU 99.3% | RAM 2.90 GB
    * GPU 2.50 / 2.58 GB -> 	Memory actively used by tensors (model weights + activations during forward). 2.58 GB = torch.cuda.memory_reserved() CUDA’s memory pool reserved for reuse (allocator buffer). how much memory PyTorch’s allocator has asked CUDA to give it so far.  2.50 / 2.58 GB is memory footprint, not utilization. ~2.5 GB = model weights + activations for one sequence. GPU memory is demand-driven, not capacity-drive, so increasing memory reserved. CORRECT WAY TO INCREASE GPU USAGE IS Use multiple processes and Each process loads: its own model copy + its own buffers (eg.) 3 processes ≈ 3 × 2.5 GB ≈ 7.5 GB VRAM. Why GPU allocated is stuck around ~2.5 GB -> torch.cuda.memory_allocated() reports live tensors currently required for computation which doesnt exceed 2.5G 
        * Method 1 to nicrease GPU: FP16. 	Keep tokens as int64 (do NOT .half() them). Only the model weights / activations go FP16. 
        * Method 2: multiple python processes, need to first split into chunks the input esm1v_input.fasta. becuase On NVIDIA GPUs (including RTX 3060), FP16 is much faster, Tensor Cores are optimized for FP16, and is numerically safe for esm1v. 
    * CPU 98–100% ->Percent of one logical core, not all cores. Bottleneck rn is the batchsize being 1. CUDA kernel launches are sequential. 	Tokenization is tiny (milliseconds). 	The GPU is already saturated by each forward pass. There is no parallel work at the Python level. So doing torch.set_num_threads(16) shouldnt speed up as CPU is not the bottleneck as its not computing the matmuls, the gpu is. 
    * RAM 2.90Gb -> This is process RSS:Using more RAM would not make this faster. GBMemory usage ≠ GPU compute utilization. CUDA kernels can be fully busy with modest memory. You are GPU compute-bound, not memory-bound. High CPU % here is normal and unavoidable in Python-driven GPU inference. GPU memory numbers show correct ESM-1v residency. Increase BATCH_SIZE → more GPU memory used, fewer kernel launches (which is what the cpu is doing)

- how to make esm1v script take more memory than 1.2Gb? On MPS, memory use is demand-driven, not user-configurable. Need to increase batch ize. 
- ESM1v with CUDA ->  700 batches/min, totlaruntime =
- esm1v with CPU -> takes 21% CPU now on btop and 3.1Gb memB. after 13min at batch 988.  after 33min its at test2500 or abtch 2500. 
- esm1v on wt+test set with MPS -> taking 1.1Gb Memb, 1.2%CPU. Took 16min. Still have free 22% memory on btop so idk why before 1h ago i had 0 free mem. ok now i got 28% with only vscode + chatgpt app + terminal btop  
* | is not recognized by wsl but is by os 

* WHY REGRESSION WEIGHTS NOT DOWNLOADING IN THE SCRIPT INSTALLING MODEL PART? If enabled (return_contacts=True), ESM produces a soft contact probability matrix:. Shape: (L × L). Value range: [0, 1]. Interpretation: probability that residues i and j are within ~8 Å (Cβ–Cβ). You can quantify how a mutation perturbs long-range residue interactions.Δ mean contacts of residue 160 . Δ contacts between loop 11 and core . Δ contact entropy. ANOTEHR USE CASE: Instead of mean-pooling residues, pool over contact-connected regions, pool active-site residues weighted by contact importance. This is not a 3D structure and not AlphaFold-level accuracy.

* for SFTP between the 2 machines need to do: lpwd, pwd, put esm1v_input.fasta, sudo nano /etc/hosts and COPY PASTE: IPADRESS bustinpc, now can do sftp justi@bustinpc easily 

* torch.cuda.empty_cache() (or torch.mps.empty_cache()) tells PyTorch: “Release any unused GPU memory that PyTorch is holding onto for reuse.”	del decides what can be freed. empty_cache() only frees already-dead tensors. 

* using what layer of esm for esm1v? -> ESM-1v is trained such that final layer is most predictive. All published ESM-1v mutation scores use final layer. Earlier layers encode syntax, not function. 

* on the first forward pass on MPS for a 650M-param transformer, This is Metal compiling and caching kernels. ESM-1v attention is O(L²), PETase ~290–310 aa → heavy. And on mac MPS does lazy kernel compilation, First invocation compiles all attention variants. This does not happen on CUDA (RTX 3060 would finish batch 1 in ~5–10 seconds).

* mean-pooled embeddings PER SEQ: One vector per sequence, Shape: (1280,), Easy to join with TSV later, Small (~5 KB per sequence)

* del tokens, reps: remove Python references → tensors can be freed. meaning temporary tensors that are computed. 

* torch.mps.empty_cache(): force PyTorch to return memory to macOS. Your Python process never keeps memory, so RSS stays ~1–2 GB even if you have 64 GB free. #WHATS RSS --> RSS = Resident Set Size. How much physical RAM the OS says your process is holding. PyTorch tensors Python objects Cached allocator memory. RSS = how much physical system RAM the OS currently assigns to your Python process.


* set_num_threads: number of threads used by intra-op CPU kernels (e.g. BLAS, MKL)
* set_num_interop_threads: number of threads used to schedule ops across CPU threads

* CPU threads are only used for:Tokenization + Python orchestration + Minor tensor setup

* cpu: CPU cores (BLAS / MKL / OpenMP), YES CAN MODIFY PYTORCH N THREADS, Matrix multiplies are multi-threaded Thread counts directly control performance. 

* mps: Apple GPU, NO. Transformer matmuls run on the GPU. If increase cpu threads to 16 while device=mps, You will see -> Slightly higher CPU usage

* cuda: NVIDIA GPU, CUDA does not use CPU threads for math, but CPU threads will be used for tokenization data loading python orchestration when running the esm1v script on windows w/ RTX gpu (meaning with CUDA)

* Attention outputs info: Contact prediction. Structure inference. Mechanistic interpretation

* per-residue esm embedding information: You want position-specific features (e.g. catalytic residues). You want to pool over regions (loops, active site). You want attention outside ESM (your own model) --> according to gpt its not needed for first-pass ranking (test set variant ranking) apparently --> Recommendation: Save mean-pooled (1280-dim) embeddings. Optionally later re-run per-residue only for top 200 variants. Note that per residue embeddings can help for: Active-site pooling Region-specific features Mechanistic feature. 

* fine-tuning means: You remove torch.no_grad(), You define a prediction head (e.g. linear layer on top of embeddings), You compute a loss (MSE / BCE / ranking loss) You call: loss.backward() and then optimizer.step() which updates ESM weights 

* ESM1v below script sequence of events (with repr33 layer): Compute embedding -> Save to disk (np.save(...)) -> Delete tensors to free memory

* ON CUDA: pip install torch --index-url https://download.pytorch.org/whl/cu121 actually pytorch packages the cuda toolkit version and cudnn version inside its wheel so the above gives 	•	CUDA runtime (e.g. cu121 = CUDA 12.1) + cuDNN + cuBLAS +	Everything needed for GPU matmul + attention. And ntoe that PyTorch wheel uses CUDA ≤12.1 while PC driver is 12.6. Writing .cu / .cpp code + Compiling with nvcc + Linking against CUDA + PyTorch + Managing GPU memory manually. PyTorch wheels already include compiled cuda kernels. 

* CUSTOM CUDA KERNELS: A CUDA kernel is a low-level GPU function written in CUDA C++ that runs directly on the GPU. Transformer attention is mathematically: QKᵀ → softmax → weighted sum; and O(L²) memory and compute. CUSTOM KERNELS implement this in different ways: Vanilla attention (slow, generic) + FlashAttention (custom CUDA kernel, fused ops, much faster) + xFormers attention (custom kernels + heuristics). AND ESM1v -> uses optimized attention kernels provided by PyTorch, On CUDA, PyTorch dispatches to highly optimized kernels. WRITING CUSTOM KERNEL BECOMES RELEVANT WHEN: Write a new architecture (not just fine-tune) + Optimize attention for very long sequences (e.g. >2k aa) + Implement physics-aware layers (e.g. distance-aware attention) + Build production inference engines + Compete with framework authors on speed. IN SUM: CUDA driver: lets GPU run code + PyTorch CUDA wheel: ships kernels + You: call models + Kernel authors: write CUDA.

* LATER ON CUDA KERNELS:  A structure-aware transformer + A protein-ligand attention model + A custom MD-aware network. eg. Build new architectures, Combine structure + docking + MD, Then CUDA kernels matter. CUDA kernels are not “cool tricks”. They are about unlocking model classes that otherwise cannot exist.  

* EG OF COMBINATION OF DOCKING MDSIM PROTEIN LIGAND ATTENTION MODEL AND STRUCTURE AWARE TRANSFORMER: 
1) Structure-aware transformers (protein sequence + 3D geometry)
	•	Mainline “structure-aware attention” is now dominated by architectures that bake in geometry either via (a) pairwise residue representations + triangle updates (AlphaFold2 / RoseTTAFold-style), or (b) SE(3)/E(n)-equivariant attention/message passing so rotations/translations behave correctly.  ￼
	•	Where the literature is going: hybrid models that jointly learn sequence priors + geometric constraints, and models that can condition on partial structures (loops, binding site) instead of full folds (useful for enzyme variants where only local regions change).  ￼

2) Protein–ligand pose models with cross-attention / equivariance
	•	Two big families:
	1.	Search/score (traditional docking + ML rescoring)
	2.	Direct pose generation with neural nets (often diffusion or equivariant regression).
	•	A current “flagship” direction is diffusion over ligand pose DOFs (translation/rotation/torsions), e.g. DiffDock, which frames docking as generative modeling and outputs confidence.  ￼
	•	Another direction is geometric deep learning that predicts binding structures using equivariant representations (example: EquiBind).  ￼

3) MD-aware networks (learning from trajectories, kinetics, enhanced sampling)
	•	“MD-aware” usually means one of:
	•	Learn collective variables / slow modes from trajectories (then do MSMs/kinetics or enhanced sampling). A classic neural approach is VAMPnets (deep learning + variational approach to Markov processes).  ￼
	•	Learn force fields / potentials (neural potentials) so you can run MD faster or with better accuracy; modern work uses equivariant architectures (e.g., TorchMD-Net family).  ￼
	•	For protein–ligand specifically: the research frontier is coupling pose uncertainty (multiple metastable binding modes) with kinetics (on/off rates) rather than just static affinity; that’s hard because you need long-timescale data or good enhanced sampling.

4) “CUDA kernels” in these projects (what people actually use them for)
	•	In practice, custom CUDA kernels matter when you need (a) faster attention, (b) faster neighbor/graph ops, (c) custom differentiable geometry, or (d) custom MD/force computations at scale. That’s why diffusion docking and equivariant geometric models tend to ship with performance-critical GPU ops.  ￼
	•	Research-grade examples of where kernels become relevant:
	•	Structure-aware transformers: fused attention / flash-attention variants, triangular updates, pairwise tensor ops.
	•	Protein–ligand attention: fast neighbor list construction, batched rigid transforms + torsion updates, equivariant message passing primitives.
	•	MD-aware models: neighbor search, long-range interactions approximations, fast equivariant tensor contractions, mixed precision stability tooling.  ￼



In [5]:
from collections import Counter

fasta_path = "data/esm1v_input.fasta"

seqs = []
with open(fasta_path) as f:
    name, buf = None, []
    for line in f:
        line = line.strip()
        if not line:
            continue
        if line.startswith(">"):
            if name:
                seqs.append((name, "".join(buf)))
            name = line[1:]
            buf = []
        else:
            buf.append(line)
    if name:
        seqs.append((name, "".join(buf)))

lengths = [len(seq) for _, seq in seqs]

print("num sequences:", len(lengths))
print("min length:", min(lengths))
print("max length:", max(lengths))
print("top 10 longest lengths:", sorted(lengths, reverse=True)[:10])

# optional: show IDs of the longest sequences
longest = sorted(seqs, key=lambda x: len(x[1]), reverse=True)[:5]
print("\nlongest sequences:")
for k, (name, seq) in enumerate(longest, 1):
    print(f"{k}. {name} length={len(seq)}")

num sequences: 4674
min length: 257
max length: 259
top 10 longest lengths: [259, 259, 259, 259, 259, 259, 259, 259, 259, 259]

longest sequences:
1. tournament_test_1|V26A length=259
2. tournament_test_2|V26C length=259
3. tournament_test_3|V26D length=259
4. tournament_test_4|V26E length=259
5. tournament_test_5|V26F length=259


In [ ]:
#CPU MEMORY STATUS  
import os, psutil, time 
proc = psutil.Process(os.getpid())

def mem_cpu():
    mem_gb = proc.memory_info().rss / (1024**3)
    cpu = proc.cpu_percent(interval=None)
    return mem_gb, cpu

#ESM1V RUNNING ON WT AND TEST SET 
import torch, esm
#torch.set_num_threads(4) #TO AVOID OVERSUBSCRIPTION
#torch.set_num_interop_threads(2) 
fasta_path = "data/esm1v_input.fasta"
BATCH_SIZE = 1   # increase to 2–4 only if it fits
device = "cpu" #or MPS FOR MAC

model, alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()
model = model.to(device).eval()
batch_converter = alphabet.get_batch_converter()

# ---------- load fasta ----------
seqs = []
with open(fasta_path) as f:
    name, buf = None, []
    for line in f:
        line = line.strip()
        if line.startswith(">"):
            if name:
                seqs.append((name, "".join(buf)))
            name = line[1:]
            buf = []
        else:
            buf.append(line)
    if name:
        seqs.append((name, "".join(buf)))

#UPDATE  
N = len(seqs)
num_batches = (N + BATCH_SIZE - 1) // BATCH_SIZE

proc = psutil.Process(os.getpid())
print(f"device: {device}")
print(f"total sequences: {N}")
print(f"batch size: {BATCH_SIZE}")
t0 = time.time()
mem0, _ = mem_cpu()

# ---------- run in batches ----------
with torch.no_grad():
    for b, i in enumerate(range(0, N, BATCH_SIZE), start=1):
        batch = seqs[i:i+BATCH_SIZE]
        batch_ids = [x[0] for x in batch]

        #MEM
        mem_before, _ = mem_cpu()
        print(mem_cpu())

        #FORWARD PASS
        _, _, tokens = batch_converter(batch)
        tokens = tokens.to(device)

        reps = model(tokens, repr_layers=[33])["representations"][33]
        # shape: (1, L, 1280)

        emb = reps[0, 1:-1].mean(dim=0).cpu().numpy()

        np.save(f"esm1v_outputs/{batch_ids[0]}.npy", emb)

        del tokens, reps, emb
        torch.mps.empty_cache()
        mem_after, cpu = mem_cpu()
        delta = mem_after - mem_before
        print(
            f"[batch {b}/{num_batches}] {batch_ids} | "
            f"RAM {mem_after:.2f} GB "
            f"(Δ {delta:+.2f} GB) | "
            f"CPU {cpu:.1f}%"
        )

        if b % 10 == 0 or b == num_batches:
            elapsed = time.time() - t0
            avg = elapsed / b
            eta = avg * (num_batches - b)
            print(
                f"  progress {100*b/num_batches:.1f}% | "
                f"elapsed {elapsed/60:.1f} min | "
                f"ETA {eta/60:.1f} min"
            )

mem_end, _ = mem_cpu()
print(f"done | peak approx RAM: {max(mem0, mem_end):.2f} GB")

/Users/bustin/.venvs/esm1v/lib/python3.11/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


device: cpu
total sequences: 4674
batch size: 1
(4.813568115234375, 0.0)
FORWARD START torch.Size([1, 261])
FORWARD END
[batch 1/4674] ['tournament_test_1|V26A'] | RAM 4.82 GB (Δ +0.00 GB) | CPU 204.3%
(4.8156280517578125, 47.7)
FORWARD START torch.Size([1, 261])
FORWARD END
[batch 2/4674] ['tournament_test_2|V26C'] | RAM 4.82 GB (Δ +0.00 GB) | CPU 215.6%
(4.8181610107421875, 144.1)
FORWARD START torch.Size([1, 261])
FORWARD END
[batch 3/4674] ['tournament_test_3|V26D'] | RAM 4.82 GB (Δ +0.00 GB) | CPU 214.4%
(4.8206634521484375, 0.0)
FORWARD START torch.Size([1, 261])
FORWARD END
[batch 4/4674] ['tournament_test_4|V26E'] | RAM 4.82 GB (Δ +0.00 GB) | CPU 216.3%
(4.821868896484375, 27.0)
FORWARD START torch.Size([1, 261])
FORWARD END
[batch 5/4674] ['tournament_test_5|V26F'] | RAM 4.82 GB (Δ +0.00 GB) | CPU 220.1%
(4.821868896484375, 118.1)
FORWARD START torch.Size([1, 261])
FORWARD END
[batch 6/4674] ['tournament_test_6|V26G'] | RAM 4.82 GB (Δ +0.00 GB) | CPU 215.9%
(4.821868896484375,

KeyboardInterrupt: 

In [ ]:
# Consolidated Alam M1–M5 classifier. Implements block-based, incremental logic anchored to the catalytic GxSxG window. Uses EXACT motif definitions from Alam et al., and this algorithm is not biologically correct  

import re
# --- Alam motif regex (exact definitions you provided, converted to Python regex) ---
M1 = re.compile(r"G.S[MLK][GS]G[GA][GAV]")
M2 = re.compile(r"G.S[MK]GGGG.{17,20}[WYF].{19,23}D.[IVLFTP]")
M3 = re.compile(r"G.S[MK]GGGG.{17,20}[WYF].{19,23}D.[IVLFTP].{27,31}[HY]")
M4 = re.compile(r"[PG]G[YF].{65,70}" + M3.pattern)

# Your M5 definition (as you wrote it)
M5 = re.compile(
    r"[PG]G[YF].{65,70}G.SMGGGG.{17,20}[WYF].{19,23}D.[IVLF].{27,31}[HY].{20,25}[ND].D.R[YF]..F.[CY]"
)

ANCHOR = re.compile(r"G.S.G")  # catalytic nucleophile anchor

def _any_match_in_window(rx, s):
    return any(rx.finditer(s))

def _m5_match_near_cterm(seq, cterm_max_tail=15):
    """
    Require that an M5 match ends within the last `cterm_max_tail` residues of the sequence.
    This prevents false M5 calls that terminate on an upstream cysteine (e.g., IsPETase C273 instead of C289).
    """
    cutoff = len(seq) - cterm_max_tail
    for m in M5.finditer(seq):
        # m.end() is 0-based exclusive; so "ends near C-term" => m.end() >= cutoff
        if m.end() >= cutoff:
            return True
    return False
def call_M_class(seq):
    """
    Returns:
        (M_class, explanation_dict)

    explanation_dict contains:
        - anchor_position
        - M1/M2/M3 evidence
        - M4 evidence (if applicable)
        - M5 evidence (if applicable)
    """

    explanation = {}

    # --- Anchor: catalytic GxSxG ---
    a = ANCHOR.search(seq)
    if not a:
        return "None", {"reason": "No GxSxG catalytic anchor found"}

    ser_idx = a.start() + 2  # S in GxSxG
    explanation["anchor_position"] = ser_idx + 1  # 1-based

    win = seq[max(0, ser_idx - 120): min(len(seq), ser_idx + 120)]

    # --- M1 ---
    m1 = M1.search(win)
    if not m1:
        return "None", {"reason": "Failed M1 (no nucleophile elbow)"}

    explanation["M1"] = {
        "matched_seq": m1.group(),
        "relative_window_span": m1.span()
    }

    # --- M2 ---
    m2 = M2.search(win)
    if not m2:
        return "M1", explanation

    explanation["M2"] = {
        "matched_seq": m2.group(),
        "relative_window_span": m2.span()
    }

    # --- M3 ---
    m3 = M3.search(win)
    if not m3:
        return "M2", explanation

    explanation["M3"] = {
        "matched_seq_tail": m3.group()[-40:],  # tail shows His region
        "relative_window_span": m3.span()
    }

    # --- M4: positional upstream clamp ---
    upstream_start = max(0, ser_idx - 70)
    upstream_end   = max(0, ser_idx - 65)
    upstream_slice = seq[upstream_start:upstream_end]

    m4 = re.search(r"[PG]G[YF]", upstream_slice)
    if not m4:
        return "M3", explanation

    explanation["M4"] = {
        "matched_seq": m4.group(),
        "absolute_positions": (
            upstream_start + m4.start() + 1,
            upstream_start + m4.end()
        ),
        "distance_from_S": ser_idx + 1 - (upstream_start + m4.start() + 1)
    }

    # --- M5 ---
    # (a) Ser+1 must be Met
    if ser_idx + 1 >= len(seq) or seq[ser_idx + 1] != "M":
        explanation["M5_fail"] = "Ser+1 is not Met"
        return "M4", explanation

    # (b) C-terminal DxDxR…C motif near end
    cterm_tail = seq[-30:]
    m5 = re.search(r"[ND].D.R[YF]..F.[CY]", cterm_tail)
    if not m5:
        explanation["M5_fail"] = "No DxDxR…C motif in C-terminal tail"
        return "M4", explanation

    explanation["M5"] = {
        "matched_seq": m5.group(),
        "absolute_positions": (
            len(seq) - 30 + m5.start() + 1,
            len(seq) - 30 + m5.end()
        ),
        "cterm_tail_used": cterm_tail
    }

    return "M5", explanation


# ---- Example ----
wt_is_aa = "MNFPRASRLMQAAVLGGLMAVSAAATAQTNPYARGPNPTAASLEASAGPFTVRSFTVSRPSGYGAGTVYYPTNAGGTVGAIAIVPGYTARQSSIKWWGPRLASHGFVVITIDTNSTLDQPSSRSSQQMAALRQVASLNGTSSSPIYGKVDTARMGVMGWSMGGGGSLISAANNPSLKAAAPQAPWDSSTNFSSVTVPTLIFACENDSIAPVNSSALPIYDSMSRNAKQFLEINGGSHSCANSGNSNQALIGKKGVAWMKRFMDNDTRYSTFACENPNSTRVSDFRTANCS"
wt_6_aa = "ASAGPFTVRSFTVSRPSGYGAGTVYYPTNAGGTVGAIAIVPGYTARQSSIKWWGPRLASHGFVVITIDTNSTLDQPSSRSSQQMAALRQVASLNGTSSSPIYGKVDTARMGVMGWSMGGGGSLISAANNPSLKAAAPQAPWDSSTNFSSVTVPTLIFACENDSIAPVNSSALPIYDSMSRNAKQFLEINGGSHSCANSGNSNQALIGKKGVAWMKRFMDNDTRYSTFACENPNSTAVSDFRTANCSLED"
wt_1_aa = "AADNPYQRGPDPTNASIEAATGPFAVGTQPIVGASGFGGGQIYYPTDTSQTYGAVVIVPGFISVWAQLNWLGPRLASQGFVVIGIETSVITDLPDPRGDQALAALDWATTRSPVASRIDRTRLAAAGWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADHFFPTSANTTMAKYFISWLKRWVDNDTRYTQFLCPGPSTGLFAPVSASMNTCPF"
wt_2_aa = "ADNPYQRGPDPTRDSVAASRGTFATASTTVGSGNGFGAGFIYYPTDTSQGTFGAVAIVPGYTATWAAEGAWMGHWLASFGFVVIGIDTINRNDWDTARGTQLLAALDYLTQRSTVRDRVDASRLAVMGHSMGGGGAMYAALQRPSLKAAVGLAPFSPSQNLNGMRVPTMLLAGQHDTTTTPASITSLYNGIPAATEKAYLELSGAGHGFPTSNNSVMMRKVIPWLKIFVDSDVRYTQFLCPLMDNTGIRSYQSTCPL"
wt_3_aa = "AQNPHERGPDPSNSYIEQARGSYSVSQRSISRLGSDGFRDGTMYYPTSTADGRFGVVAISPGYTASESTIAWLGPRLASFGFVVVTINTDSRYDQPRQRATQLHAALDHAIGDSVVGPRIDTSRQAVMGHSMGGGGALQAAEERDEIRAAVPLTPWNLKKGWSGVDAATLVIGAENDAIAPVRSHSIPFYESLTNAERRAYLELRREGHFAPNSSNTLIAKYSVSWLKRYVDNDLRYDQFIDPGPRTGITTGVSDYRLG"

print(call_M_class(wt_is_aa))
print(call_M_class(wt_1_aa))
print(call_M_class(wt_2_aa))
print(call_M_class(wt_3_aa))
print(call_M_class(wt_6_aa))

('M3', {'anchor_position': 160, 'M1': {'matched_seq': 'GWSMGGGG', 'relative_window_span': (118, 126)}, 'M2': {'matched_seq': 'GWSMGGGGSLISAANNPSLKAAAPQAPWDSSTNFSSVTVPTLIFACENDSI', 'relative_window_span': (118, 169)}, 'M3': {'matched_seq_tail': 'TLIFACENDSIAPVNSSALPIYDSMSRNAKQFLEINGGSH', 'relative_window_span': (118, 198)}})
('M4', {'anchor_position': 129, 'M1': {'matched_seq': 'GWSMGGGG', 'relative_window_span': (118, 126)}, 'M2': {'matched_seq': 'GWSMGGGGLRRAALQRPSLKAIVGMAPWNGERNWSAVTVPTLFFGGSSDAV', 'relative_window_span': (118, 169)}, 'M3': {'matched_seq_tail': 'TLFFGGSSDAVASPNDHAKPFYNSITRAEKDYIELRNADH', 'relative_window_span': (118, 198)}, 'M4': {'matched_seq': 'PGF', 'absolute_positions': (59, 61), 'distance_from_S': 70}, 'M5_fail': 'No DxDxR…C motif in C-terminal tail'})
('M3', {'anchor_position': 130, 'M1': {'matched_seq': 'GHSMGGGG', 'relative_window_span': (118, 126)}, 'M2': {'matched_seq': 'GHSMGGGGAMYAALQRPSLKAAVGLAPFSPSQNLNGMRVPTMLLAGQHDTT', 'relative_window_span': (118, 169

# 1. **TEST SET ANALYSIS JAN 6**

**Annotate the test set by mutation type**

* The file reveals that the test set is not a random error-prone library, but a systematic Site Saturation Mutagenesis (SSM) library - specific residues (e.g., Q100, P113, I118, W154, W160) substituted with every other amino acid (A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y).

* Mutation types to screen (PETase-specific and family-agnostic):
    * A: Cys/Disulfide Liability
    * B: Active Site & Substrate Binding
    * C: Surface Charge/Solubility Remodeling
    * D: pH robustness 
    * E Loop Rigidification
    * F Aromatic Clamp / W-loop wobbling
    * G Local Salt-Bridge / H-bond rewiring 
    * H Hydrophobic Core Packing 
    * I Oxyanion Hole Geometry Change 
    * J M5 + family motif integrity 
    * K Electrostatic Field Tuning 
    * L Metal/Ca Coordination Weakening 

* In IsPETase: 
* In CaPETase (wt1):
* In wt2:
* In wt3: 


In [ ]:
import pandas as pd
import re

# --- amino acid properties ---
AA_CLASS = {
    # hydrophobic
    "A": "hydrophobic", "V": "hydrophobic", "I": "hydrophobic",
    "L": "hydrophobic", "M": "hydrophobic", "F": "hydrophobic",
    "W": "hydrophobic", "Y": "hydrophobic",
    # polar
    "S": "polar", "T": "polar", "N": "polar", "Q": "polar",
    # charged
    "D": "charged", "E": "charged", "K": "charged", "R": "charged", "H": "charged",
    # special
    "G": "special", "P": "special", "C": "special"
}

AA_CHARGE = {
    "D": -1, "E": -1,
    "K": +1, "R": +1,
    "H": +0.5,  # pH-dependent, keep explicit
}

AA_VOLUME = {  # rough vdW volume bins
    "G": 1, "A": 2, "S": 3, "C": 3, "P": 3,
    "T": 4, "D": 4, "N": 4,
    "V": 5, "E": 5, "Q": 5,
    "H": 6, "I": 6, "L": 6, "M": 6,
    "F": 7, "Y": 7,
    "K": 8, "R": 9, "W": 10
}

AROMATIC = {"F", "Y", "W"}

def annotate_mutation(mut):
    m = re.match(r"([A-Z])(\d+)([A-Z])", mut)
    if not m:
        return None

    wt, pos, mt = m.groups()
    pos = int(pos)

    wt_class = AA_CLASS.get(wt, "unknown")
    mt_class = AA_CLASS.get(mt, "unknown")

    wt_charge = AA_CHARGE.get(wt, 0)
    mt_charge = AA_CHARGE.get(mt, 0)

    delta_charge = mt_charge - wt_charge

    wt_vol = AA_VOLUME.get(wt, 0)
    mt_vol = AA_VOLUME.get(mt, 0)

    if mt_vol > wt_vol:
        size_change = "larger"
    elif mt_vol < wt_vol:
        size_change = "smaller"
    else:
        size_change = "similar"

    conservative = (
        wt_class == mt_class and abs(delta_charge) == 0
    )

    return {
        "wt_aa": wt,
        "mut_aa": mt,
        "position": pos,
        "wt_class": wt_class,
        "mut_class": mt_class,
        "polarity_change": f"{wt_class}->{mt_class}",
        "wt_charge": wt_charge,
        "mut_charge": mt_charge,
        "delta_charge": delta_charge,
        "size_change": size_change,
        "is_gly_gain": int(mt == "G" and wt != "G"),
        "is_gly_loss": int(wt == "G" and mt != "G"),
        "is_pro_gain": int(mt == "P" and wt != "P"),
        "is_pro_loss": int(wt == "P" and mt != "P"),
        "is_cys_gain": int(mt == "C" and wt != "C"),
        "is_cys_loss": int(wt == "C" and mt != "C"),
        "aromatic_gain": int(mt in AROMATIC and wt not in AROMATIC),
        "aromatic_loss": int(wt in AROMATIC and mt not in AROMATIC),
        "conservative": int(conservative),
    }

# --- usage ---
# df must contain a column named "mutation"
def annotate_dataframe(df):
    ann = df["mutation"].apply(annotate_mutation)
    ann_df = pd.DataFrame(list(ann))
    return pd.concat([df.reset_index(drop=True), ann_df], axis=1)

# Example:
# df = pd.read_csv("input.tsv", sep="\t")
# df_out = annotate_dataframe(df)
# df_out.to_csv("annotated.tsv", sep="\t", index=False)

**Charlie's feature extraction script**

In [65]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm

print("Libraries imported")
# ============================================================
# Amino Acid Properties
# ============================================================
AMINO_ACIDS = list("ACDEFGHIKLMNPQRSTVWY")

# Kyte-Doolittle hydropathy scale
HYDROPATHY = {
    'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8,
    'G': -0.4, 'H': -3.2, 'I': 4.5, 'K': -3.9, 'L': 3.8,
    'M': 1.9, 'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5,
    'S': -0.8, 'T': -0.7, 'V': 4.2, 'W': -0.9, 'Y': -1.3
}

# Molecular weight
MOLECULAR_WEIGHT = {
    'A': 89.1, 'C': 121.2, 'D': 133.1, 'E': 147.1, 'F': 165.2,
    'G': 75.1, 'H': 155.2, 'I': 131.2, 'K': 146.2, 'L': 131.2,
    'M': 149.2, 'N': 132.1, 'P': 115.1, 'Q': 146.2, 'R': 174.2,
    'S': 105.1, 'T': 119.1, 'V': 117.1, 'W': 204.2, 'Y': 181.2
}

# Volume (Angstrom^3)
VOLUME = {
    'A': 88.6, 'C': 108.5, 'D': 111.1, 'E': 138.4, 'F': 189.9,
    'G': 60.1, 'H': 153.2, 'I': 166.7, 'K': 168.6, 'L': 166.7,
    'M': 162.9, 'N': 114.1, 'P': 112.7, 'Q': 143.8, 'R': 173.4,
    'S': 89.0, 'T': 116.1, 'V': 140.0, 'W': 227.8, 'Y': 193.6
}

# Charge at pH 7
CHARGE = {
    'A': 0, 'C': 0, 'D': -1, 'E': -1, 'F': 0,
    'G': 0, 'H': 0.5, 'I': 0, 'K': 1, 'L': 0,
    'M': 0, 'N': 0, 'P': 0, 'Q': 0, 'R': 1,
    'S': 0, 'T': 0, 'V': 0, 'W': 0, 'Y': 0
}

# Polarity
POLARITY = {
    'A': 0, 'C': 0, 'D': 1, 'E': 1, 'F': 0,
    'G': 0, 'H': 1, 'I': 0, 'K': 1, 'L': 0,
    'M': 0, 'N': 1, 'P': 0, 'Q': 1, 'R': 1,
    'S': 1, 'T': 1, 'V': 0, 'W': 0, 'Y': 1
}

print("Amino acid properties defined")

# ============================================================
# Feature Extraction Functions
# ============================================================

def compute_aa_composition(sequence):
    """Compute amino acid composition"""
    length = len(sequence)
    counts = Counter(sequence)
    return {f'comp_{aa}': counts.get(aa, 0) / length for aa in AMINO_ACIDS}

def compute_physicochemical_features(sequence):
    """Compute physicochemical properties"""
    length = len(sequence)

    hydro = [HYDROPATHY.get(aa, 0) for aa in sequence]
    mw = [MOLECULAR_WEIGHT.get(aa, 0) for aa in sequence]
    vol = [VOLUME.get(aa, 0) for aa in sequence]
    charge = [CHARGE.get(aa, 0) for aa in sequence]
    polar = [POLARITY.get(aa, 0) for aa in sequence]

    features = {
        'length': length,
        'hydropathy_mean': np.mean(hydro),
        'hydropathy_std': np.std(hydro),
        'mw_total': np.sum(mw),
        'mw_mean': np.mean(mw),
        'volume_mean': np.mean(vol),
        'charge_total': np.sum(charge),
        'charge_positive': sum(1 for c in charge if c > 0),
        'charge_negative': sum(1 for c in charge if c < 0),
        'polarity_ratio': np.mean(polar),
        'gravy': np.sum(hydro) / length,  # GRAVY score
    }
    return features

def compute_regional_features(sequence, n_regions=4):
    """Compute features for different regions"""
    length = len(sequence)
    region_size = length // n_regions

    features = {}
    for i in range(n_regions):
        start = i * region_size
        end = start + region_size if i < n_regions - 1 else length
        region = sequence[start:end]

        hydro = [HYDROPATHY.get(aa, 0) for aa in region]
        charge = [CHARGE.get(aa, 0) for aa in region]

        features[f'region{i+1}_hydropathy'] = np.mean(hydro) if hydro else 0
        features[f'region{i+1}_charge'] = np.sum(charge) if charge else 0

    return features

def compute_terminal_features(sequence, window=10):
    """Compute N-terminal and C-terminal features"""
    n_term = sequence[:window]
    c_term = sequence[-window:]

    return {
        'nterm_hydropathy': np.mean([HYDROPATHY.get(aa, 0) for aa in n_term]),
        'nterm_charge': sum(CHARGE.get(aa, 0) for aa in n_term),
        'cterm_hydropathy': np.mean([HYDROPATHY.get(aa, 0) for aa in c_term]),
        'cterm_charge': sum(CHARGE.get(aa, 0) for aa in c_term),
    }

def extract_all_features(sequence):
    """Extract all features for a sequence"""
    features = {}
    features.update(compute_aa_composition(sequence))
    features.update(compute_physicochemical_features(sequence))
    features.update(compute_regional_features(sequence))
    features.update(compute_terminal_features(sequence))
    return features

print("Feature extraction functions defined")

# ============================================================
# Extract Mutation Features
# ============================================================

def parse_mutation_string(mutation_str):
    """Parse mutation string like 'A26C;G100V' to list of tuples"""
    if pd.isna(mutation_str) or mutation_str == '':
        return []

    mutations = []
    for mut in mutation_str.split(';'):
        if len(mut) >= 3:
            wt_aa = mut[0]
            mut_aa = mut[-1]
            pos = int(mut[1:-1]) - 1  # Convert to 0-indexed
            mutations.append((pos, wt_aa, mut_aa))
    return mutations

def compute_mutation_features(mutation_str, seq_length):
    """Compute features based on mutations"""
    mutations = parse_mutation_string(mutation_str)

    if len(mutations) == 0:
        return {
            'num_mutations': 0,
            'mutation_position_norm': 0,
            'delta_hydropathy': 0,
            'delta_volume': 0,
            'delta_charge': 0,
            'delta_polarity': 0,
            'is_wildtype': 1,
        }

    # Mutation position (normalized)
    positions = [m[0] for m in mutations]
    pos_norm = np.mean(positions) / seq_length

    # Delta properties
    delta_hydro = sum(HYDROPATHY.get(m[2], 0) - HYDROPATHY.get(m[1], 0) for m in mutations)
    delta_vol = sum(VOLUME.get(m[2], 0) - VOLUME.get(m[1], 0) for m in mutations)
    delta_charge = sum(CHARGE.get(m[2], 0) - CHARGE.get(m[1], 0) for m in mutations)
    delta_polar = sum(POLARITY.get(m[2], 0) - POLARITY.get(m[1], 0) for m in mutations)

    return {
        'num_mutations': len(mutations),
        'mutation_position_norm': pos_norm,
        'delta_hydropathy': delta_hydro,
        'delta_volume': delta_vol,
        'delta_charge': delta_charge,
        'delta_polarity': delta_polar,
        'is_wildtype': 0,
    }

print("Mutation feature functions defined")

# ============================================================
# Extract Features for All Sequences
# ============================================================
print("Extracting features for all sequences...")

all_features = []

for i in tqdm(range(len(test_df)), desc="Extracting features"):
    seq = test_df.iloc[i]['sequence']

    # Sequence features
    features = extract_all_features(seq)

    # Mutation features
    mutation_str = mutation_df.iloc[i]['mutations_str']
    mut_features = compute_mutation_features(mutation_str, len(seq))
    features.update(mut_features)

    # Wildtype index
    features['wt_idx'] = mutation_df.iloc[i]['wt_idx']

    all_features.append(features)

# Convert to DataFrame
features_df = pd.DataFrame(all_features)
print(f"\nFeature matrix shape: {features_df.shape}")
print(f"Number of features: {len(features_df.columns)}")

ModuleNotFoundError: No module named 'tqdm'

# 2. **Benchmark Datasets Integration**

In [15]:
os.chdir("..")

# **2. Loading external stability and expression datasets**

**NESG Solubility** 
(https://loschmidt.chemi.muni.cz/soluprot/?page=download)
* 10k proteins
* Labels: exp, sol, uniprot id or local ID 
* Units: integer 

**Soluprot Solubility**
(https://loschmidt.chemi.muni.cz/soluprot/?page=download)
* 11k training, 3k test
* Label: solubility, number IDs with no conversion map (has seq)
* Unit: 0/1

**Price Solubility**
(https://pmc.ncbi.nlm.nih.gov/articles/PMC3372292/)
* 7k proteins 
* Label: usability. uniprot id
* Unit: 0/1

**PSI Solubility** 
(https://academic.oup.com/bioinformatics/article/36/18/4691/5860015?login=false)
* 11k proteins
* Label: solubility, Aa0000 ID scheme (has seq)
* Unit: 0/1
* Note: ecoli with custom IDs, dropped for now 

**Meltome Stability** 
(https://meltomeatlas.proteomics.wzw.tum.de/master_meltomeatlasapp/)
* 1M variants 
* Label: temperature, meltpoint, fold_change, uniprot id 
* Note: ecoli with custom IDs, dropped for now  

**FireprotDB Stability** 
(https://loschmidt.chemi.muni.cz/fireprotdb/)
* 53k variants
* Label: ddG, dTm, pH, Tm, mutation_effect, uniprot id 

**ThermomutDB Stability**
(https://biosig.lab.uq.edu.au/thermomutdb/downloads)
* 12k variants
* Label: pH, ddG, temperature, dTm, uniprot/pdb id 
* Note: these genes were not retrieved from the database due to removal from uniprotkb:  A0A410ZNC6 D0WVP7 G7LSK3 GQ884175 M5A5Y8 Q9REI6

**CAFA** 
(https://www.kaggle.com/competitions/cafa-5-protein-function-prediction/code)
* 142k variants

**Novozyme**
(https://www.kaggle.com/code/jinyuansun/eda-and-finetune-esm)
* 31k variants

**Protsol Solubility**
(https://huggingface.co/datasets/AI4Protein/ProtSolM)
* 71k proteins
* Label: solubility, no ID but has sequence
* Unit: 0/1 

In [ ]:
#we will now load "masterdb.csv" found under data
import pandas as pd 
import os 
path = "data/masterdb.tsv"
df = pd.read_csv(path,sep="\t")
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("?", "")
)
df2 = pd.DataFrame()
df2["id"]=df["name"]
df2["sequence"] = df["protein_sequence"].astype(str)
print(df2)

def dftofasta(df,outfile):
    with open(outfile,"w") as f:
        for index,row in df.iterrows():
            f.write(f">{row['id']}\n")
            f.write(f"{row['sequence']}\n")
    return outfile 

#dftofasta(df2,"data/masterdb.fasta")

In [112]:
############################################
#Setting up functions 
############################################
import pandas as pd
import json
import numpy as np
from pathlib import Path
import requests
from Bio.SeqIO.FastaIO import SimpleFastaParser

def read_fasta_dict(path: str):
    seqs = {}
    with open(path) as fh:
        for header, seq in SimpleFastaParser(fh):
            sid = header.split()[0].strip()
            seqs[sid] = seq.strip()
    return seqs

def load_nesg(csv_path: str, fasta_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)  # uses CSV header row directly: id, exp, sol
    seqs = read_fasta_dict(fasta_path)
    #"sid" "usability" "fasta" 
    df["sequence"] = df["id"].map(seqs)
    return df 

def load_psi(csv_path: str, psi_detail_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)                        # has sid + fasta + labels
    psi_all = pd.read_csv(psi_detail_path, sep="\t")  # extra metadata

    # merge on sid
    df = df.merge(psi_all, on="sid", how="left")

    # sequence is already in the "fasta" column
    df["sequence"] = df["fasta"]

    return df
def load_price(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    df["sequence"] = df["fasta"]
    # "sid" "usability" "fasta" 
    return df

def load_soluprot(train_csv: str, test_csv: str,
                  train_fasta_path: str, test_fasta_path: str) -> pd.DataFrame:

    # load FASTA → dict, keys exactly as in FASTA headers
    train_fasta = read_fasta_dict(train_fasta_path)
    test_fasta  = read_fasta_dict(test_fasta_path)

    # merge FASTA dicts
    fasta = {**train_fasta, **test_fasta}

    # load CSVs
    df1 = pd.read_csv(train_csv)
    df2 = pd.read_csv(test_csv)

    df = pd.concat([df1, df2], ignore_index=True)

    # map sequences using the exact ids
    df["sequence"] = df["sid"].map(fasta)


    return df


def load_meltome(csv_path: str, fasta_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # load fasta into dict: {uniprot_id: sequence}
    fasta = read_fasta_dict(fasta_path)

    # extract uniprot prefix from Protein_ID (before "_")
    df["uniprot_id"] = df["Protein_ID"].astype(str).apply(lambda x: x.split("_")[0])

    # map sequences
    df["sequence"] = df["uniprot_id"].map(fasta)

    return df

def load_fireprot(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # "uniprot_id" "pdb_id" "muutation" "ddG" "dTm" "pH" "tm" "mutation_effect" "sequence"
    return df 

def load_thermomut(json_path: str, fasta_path: str) -> pd.DataFrame:
    # load JSON metadata
    with open(json_path) as fh:
        data = json.load(fh)
    df = pd.DataFrame(data)

    # load FASTA sequences
    fasta_dict = read_fasta_dict(fasta_path)

    # map UniProt → sequence
    # JSON column is "uniprot"
    df["sequence"] = df["uniprot"].map(fasta_dict)

    # ensure required labels exist even if missing in JSON
    for col in ["ph","ddg","temperature","dtm","PDB_wild",
                "pdb_mutant","mutation_code","mutated chain","effect"]:
        if col not in df.columns:
            df[col] = None

    return df

def load_protsolm(train_csv: str, test_csv: str) -> pd.DataFrame:
    df1 = pd.read_csv(train_csv)
    df2 = pd.read_csv(test_csv)
    df = pd.concat([df1, df2], ignore_index=True)
    # "aa_seq" "detail"
    return df

def fasta_merger(fasta_paths: list, outpath: str):
    with open(outpath, "w") as out:
        for path in fasta_paths:
            with open(path) as fh:
                for line in fh:
                    out.write(line)

def parse_cd_hit_clusters(clstr_path):
    clusters = {}
    current = None

    with open(clstr_path) as f:
        for line in f:
            line = line.strip()
            if line.startswith(">Cluster"):
                current = line.split()[1]
                clusters[current] = []
            else:
                # Example: "0       50aa, >SEQ123... *"
                sid = line.split(">")[1].split("...")[0]
                clusters[current].append(sid)

    return clusters

def fasta_merger_from_dfs(datasets, outpath):
    with open(outpath, "w") as out:
        for i, df in enumerate(datasets):
            for idx, row in df.iterrows():
                seq = row["sequence"]
                if seq is None or pd.isna(seq):
                    continue
                header = f"{df.__class__.__name__}_{i}_{idx}"
                out.write(f">{header}\n{seq}\n")

def fetch_uniprot_fasta(ids, out_fasta, delay=0.15):
    """Fetch FASTA for many UniProt IDs with error handling."""
    with open(out_fasta, "w") as out:
        for uid in ids:
            url = f"https://rest.uniprot.org/uniprotkb/{uid}.fasta"
            r = requests.get(url, timeout=10)

            if r.status_code == 200 and r.text.startswith(">"):
                out.write(r.text.strip() + "\n")
            else:
                # write placeholder for failed fetch
                out.write(f">{uid}\nFAILED_FETCH\n")

            time.sleep(delay)  # rate-limit to avoid 500 errors



def load_novozymes(train_path: str, test_path: str, test_labels_path: str) -> pd.DataFrame:
    train_df = pd.read_csv(train_path)         # seq_id, protein_sequence, pH, Tm
    test_df = pd.read_csv(test_path)           # seq_id, protein_sequence, pH
    test_labels = pd.read_csv(test_labels_path)  # seq_id, Tm

    # merge test with its labels
    test_df = test_df.merge(test_labels, on="seq_id", how="left")

    # unify column names to match your other datasets
    train_df["sequence"] = train_df["protein_sequence"]
    test_df["sequence"]  = test_df["protein_sequence"]

    # combine train + test into one dataframe
    df = pd.concat([train_df, test_df], ignore_index=True)

    return df

In [ ]:
############################################
#setting the paths, loading them into dataframes, and making the merged fasta file to cdhit 
############################################


#PSI_PATH      = "data/benchmark/sol_benchmark/PSI_Biology_solubility_trainset.csv"
#psi_detail_path = "data/benchmark/sol_benchmark/PSI_all_data_esol.tab"
NESG_PATH     = "data/benchmark/sol_benchmark/nesg/nesg.csv"
nesg_fasta_path = "data/benchmark/sol_benchmark/nesg/nesg.fasta"
PRICE_PATH    = "data/benchmark/sol_benchmark/Price_usability_trainset.csv"
soluprot_train_path = "data/benchmark/sol_benchmark/soluprot_data/training_set.csv"
soluprot_test_path = "data/benchmark/sol_benchmark/soluprot_data/test_set.csv" 
soluprot_train_fasta = "data/benchmark/sol_benchmark/soluprot_data/training_set.fasta"
soluprot_test_fasta = "data/benchmark/sol_benchmark/soluprot_data/test_set.fasta" 
#meltome_path = "data/benchmark/stab_benchmark/meltome_cross-species.csv"
#meltome_fasta_path = "data/benchmark/stab_benchmark/meltome_fasta.fasta"
fireprot_path = "data/benchmark/stab_benchmark/fireprotdb_results_stability.csv"
thermomutdb_path = "data/benchmark/stab_benchmark/thermomutdb.json"
thermomutdb_fasta = "data/benchmark/stab_benchmark/thermomutdb.fasta"
protsol_train_path = "data/benchmark/protsolm_data/protsolm_train.csv"
protsol_test_path = "data/benchmark/protsolm_data/protsolm_test.csv"
novozyme_test_path = "data/benchmark/novozymes-enzyme-stability-prediction/test.csv"
novozyme_train_path = "data/benchmark/novozymes-enzyme-stability-prediction/train.csv"
novozyme_test_labels_path = "data/benchmark/novozymes-enzyme-stability-prediction/test_labels.csv"


#LOAD DATASETS 
#psi = load_psi("data/benchmark/sol_benchmark/PSI_Biology_solubility_trainset.csv","data/benchmark/sol_benchmark/PSI_all_data_esol.tab")
nesg = load_nesg(NESG_PATH, nesg_fasta_path) #no seq col 
price = load_price(PRICE_PATH) #fasta 
soluprot = load_soluprot(
    soluprot_train_path,
    soluprot_test_path,
    soluprot_train_fasta,
    soluprot_test_fasta    
)
fireprot = load_fireprot(fireprot_path) #sequence
thermomut = load_thermomut(thermomutdb_path,thermomutdb_fasta)
#meltome = load_meltome(meltome_path,meltome_fasta_path) 
protsolm = load_protsolm(protsol_train_path, protsol_test_path) #aa_seq
novozymes = load_novozymes(
    novozyme_train_path, novozyme_test_path, novozyme_test_labels_path
)

#ADD FASTA TO DATAFRAMES IF THE CSV DID NOT HAVE IT
nesg_fasta = read_fasta_dict(nesg_fasta_path)
nesg["sequence"] = nesg["id"].map(nesg_fasta)
price["sequence"] = price["fasta"]
protsolm["sequence"] = protsolm["aa_seq"]


#MERGE ALL FASTA FILES FOR CD-HIT
def fasta_merger_from_dfs(datasets, outpath):
    with open(outpath, "w") as out:
        for i, df in enumerate(datasets):
            for idx, row in df.iterrows():
                seq = row["sequence"]
                if seq is None or pd.isna(seq):
                    continue
                header = f"{df.__class__.__name__}_{i}_{idx}"
                out.write(f">{header}\n{seq}\n")

datasets = [nesg, price, soluprot, fireprot, protsolm,thermomut, novozymes]
fasta_merger_from_dfs(datasets, "allbenchmarks.fasta")

#SETTING  CANONICAL IDS
for i, df in enumerate(datasets):
    df["canonical_id"] = [
        f"{df.__class__.__name__}_{i}_{idx}" for idx in df.index
    ]

/var/folders/y8/spmd84w16l78w44w6_yz5h340000gn/T/ipykernel_28883/3962239254.py:81: DtypeWarning: Columns (23,24,25,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [ ]:
############################################
#Running CD-HIT, parsing output and merging the final merged_df
############################################

#cd-hit -i all_sequences.fasta -o all_sequences_100.fasta -c 1.0 -n 5 -d 0
#PARSING THE CLUSTERS FROM CD-HIT INTO ONE MERGED DATAFRAME 
datasets = {
    "nesg": nesg,
    "price": price,
    "soluprot": soluprot,
    "fireprot": fireprot,
    "protsolm": protsolm,
    "thermomutdb":thermomut,
    "novozymes": novozymes
}

# Ensure every df has canonical_id as previously assigned
for name, df in datasets.items():
    if "canonical_id" not in df.columns:
        raise ValueError(f"{name} missing canonical_id")


clusters = parse_cd_hit_clusters("data/benchmark/benchmark_cdhit100_cluster.txt")
merged_rows = []
for clust_id, members in clusters.items():
    rep = members[0]
    collected = []
    for name, df in datasets.items():
        sub = df[df["canonical_id"].isin(members)]
        if len(sub) > 0:
            sub = sub.copy()
            sub["source_dataset"] = name
            collected.append(sub)

    if collected:
        merged_block = pd.concat(collected, ignore_index=True)
    else:
        merged_block = pd.DataFrame()

    merged_rows.append({
        "cluster_id": clust_id,
        "canonical_ids": members,
        "representative": rep,
        "merged_block": merged_block
    })

# this is your final merged output
merged_df = pd.DataFrame(merged_rows)

print(merged_df.shape)
merged_df.head()

In [114]:
KEEP_COLS = {
    "fireprot": [
        "uniprot_id","pdb_id","chain","wild_type","position","mutation",
        "pH","tm","dTm","ddG","interpro_families","is_essential","sequence"
    ],
    "protsolm": ["name","label","detail","sequence"],
    "novozymes": ["seq_id","pH","tm","protein_sequence","sequence"],
    "nesg": ["sid","exp","sol","solubility","fasta","sequence"],
    "price": ["sid","Usability|0=NotUsable|1=Usable","fasta","sequence"]
}

def clean_block(df):
    if df.empty:
        return df
    src = df["source_dataset"].iloc[0]
    if src not in KEEP_COLS:
        return df
    keep = KEEP_COLS[src]
    # always preserve canonical_id + source_dataset
    keep = [c for c in keep if c in df.columns] + ["canonical_id","source_dataset"]
    return df[keep]

merged_df["merged_block"] = merged_df["merged_block"].apply(clean_block)

In [ ]:
############################################################
# Attempt to get structure of this benchmark database to make multimodal db but will need to filter the proteins
############################################################

missing_pdb_uniprot = set()
has_pdb_ids = set()
no_uniprot_no_pdb_seqs = set()

for block in merged_df["merged_block"]:
    if not isinstance(block, pd.DataFrame) or block.empty:
        continue

    cols = block.columns

    # 1. uniprot_id present but pdb_id missing
    if "uniprot_id" in cols:
        b = block[["uniprot_id", "pdb_id"]].copy()
        b = b[b["uniprot_id"].notna()]  # entries with uniprot
        b = b[b["pdb_id"].isna()]       # missing pdb
        missing_pdb_uniprot.update(b["uniprot_id"].dropna().astype(str).tolist())

    # 2. unique pdb ids
    if "pdb_id" in cols:
        ids = block["pdb_id"].dropna().astype(str).tolist()
        has_pdb_ids.update(ids)

    # 3. no uniprot, no pdb → need sequences
    if "sequence" in cols:
        mask = pd.Series(True, index=block.index)
        if "uniprot_id" in cols:
            mask &= block["uniprot_id"].isna()
        if "pdb_id" in cols:
            mask &= block["pdb_id"].isna()
        seqs = block.loc[mask, "sequence"].dropna().tolist()
        for s in seqs:
            no_uniprot_no_pdb_seqs.add(s)


############################################################
# WRITE OUTPUT FILES
############################################################

# 1. uniprot present but no pdb
with open("missing_pdb_uniprot_ids.txt", "w") as f:
    for uid in sorted(missing_pdb_uniprot):
        f.write(uid + "\n")

# 2. pdb ids
with open("has_pdb_ids.txt", "w") as f:
    for pid in sorted(has_pdb_ids):
        f.write(pid + "\n")

# 3. fasta for no uniprot AND no pdb
with open("no_uniprot_no_pdb.fasta", "w") as f:
    for i, seq in enumerate(sorted(no_uniprot_no_pdb_seqs)):
        f.write(f">seq_{i}\n{seq}\n")

print("DONE:")
print(" missing_pdb_uniprot_ids.txt")
print(" has_pdb_ids.txt")
print(" no_uniprot_no_pdb.fasta")


#Run mmseqs2 to seq2struct to find top hits, if not, use AF2

In [120]:
all_pdb_ids = set()

for block in merged_df["merged_block"]:
    if isinstance(block, pd.DataFrame) and "pdb_id" in block.columns:
        all_pdb_ids.update(block["pdb_id"].dropna().unique())
print(all_pdb_ids)

{'1FEP', '1UHG', '1AZP|1AZP|1AZP|1BF4', '1IMQ', '3VUB|3VUB', '2ADA', '1BNI|1A2P', '1AXB|1BTL|1XPB|1ZG4', '1YPI|1YPI', '1IDS|1IDS|1IDS|1IDS', '2TRX', '2IMM', '1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ|1ZNJ', '1CYC', '1CAH', '1DPM', '1B5M', '1UZC', '3MBP|1SVX|1SVX', '3PG0', '1RX4|1DDR|1DDR|1DYJ|5DFR', '1DKT|1DKT', '1HME', '1C52', '1P2P|1P2P', '1AAR|1AAR', '1PDO', '1JU3', '1LBI|1LBI|1LBI|1LBI', '1RIS|1RIS|1RIS|1RIS', '1C9O|1C9O', '3BLS|1KE4', '1YYX', '1BNL', '3HHR|1HGU', '1KCQ', '1QGD|1QGD', '1E21', '1LVE', '1BCX', '1TIN', '1ROP|1ROP', '1ZYM', '1ARR|1ARR', '1HNG|1HNG|1CDC|1CDC', '1I4N', '1RTP', '1TPK', '1IR3|1IR3|1IR3|1IR3', '1TTG|1FNA|1FNF', '1APS', '1BP2|1G4I', '3D2A|1ISP|3D2C', '1CF3', '1HK0', '1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON|1AON', '1WIT', '1M21|1M21', '1CYO', '8TIM', '451C', '1KFW', '2Q98', '4ZLU|4ZLU', '2ABD', '3PGK', '3WP4', '1LUC', '1DIL|3SIL', '2CBR', '1OLR', '1BAH', '1HZ6', '1AM7|1AM7|1AM7', '1K9Q|1